# Grid Search

Proviamo a risolvere lo stesso problema precedente attraverso algoritmi di grid search per la scelta della rete neurale migliore. E' una ricerca molto più dispendiosa a livello di uso di memoria in quanto vengono testate e validate diverse reti.

## Importazione delle librerie usate con le relative versioni

In [13]:
import numpy
print('numpy:', numpy.__version__)

import scipy
print('scipy:', scipy.__version__)

import matplotlib
print('matplotlib:', matplotlib.__version__)

import IPython
print('iPython:', IPython.__version__)

import sklearn
print('scikit-learn:', sklearn.__version__)

import tensorflow
print('tensorflow:', tensorflow.__version__)

import keras
print('keras:', keras.__version__) 

numpy: 1.15.4
scipy: 1.1.0
matplotlib: 3.0.1
iPython: 7.2.0
scikit-learn: 0.20.1
tensorflow: 1.12.0
keras: 2.2.4


In [31]:
import pylab as pl
import numpy as np  
import pandas as pd
import argparse
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d
import scipy.odr as odr 
from keras.activations import relu
from keras.activations import tanh
import matplotlib.pyplot as plt 
import matplotlib as mp
import itertools
import random
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from array import array
from sklearn import metrics
from keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn import svm, datasets
from scipy import interp
from keras.callbacks import History 
history = History()
from keras.callbacks import *
from sklearn.externals import joblib
from scipy.stats import norm
from sklearn.model_selection import GridSearchCV
import scipy.stats
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.metrics import mean_absolute_error
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.layers.normalization import BatchNormalization

Importiamo i dati su cui performare la regressione e dividiamo il dataset in training set validation set e test set:

In [9]:
ewk_input = np.load('ewk_signal_withreco_withdelta_input.npy')
ewk_truth = np.load('ewk_signal_withreco_withdelta_truth.npy')
ewk_input_train = ewk_input[0:400000,:]
ewk_truth_train = ewk_truth[0:400000,4].reshape(-1, 1)

ewk_input_validation = ewk_input[400000:450000,:]
ewk_truth_validation = ewk_truth[400000:450000,4].reshape(-1, 1)

ewk_input_test = ewk_input[450000:501328,:]
ewk_truth_test = ewk_truth[450000:501328,4].reshape(-1, 1)

Z-scaliamo il dataset in modo da ottenere dati più maneggiabili poichè sono più vicini allo 0.
Z-scalare vuol dire fare la seguente cosa:
$ data_{new} = \frac{data_{old}-\mu}{\sigma} $
Salviamo successivamente il modello scalato in un file 'scaler_input.pkl' in modo da poterlo richiamare successivamente

In [11]:
scaler_input = StandardScaler()
scaler_truth = StandardScaler()


scaler_input.fit(ewk_input_train)
scaler_truth.fit(ewk_truth_train)

input_train = scaler_input.transform(ewk_input_train)
truth_train = scaler_truth.transform(ewk_truth_train)
input_validation = scaler_input.transform(ewk_input_validation)
truth_validation = scaler_truth.transform(ewk_truth_validation)
input_test = scaler_input.transform(ewk_input_test)
truth_test = ewk_truth_test

joblib.dump(scaler_input,  "scaler_input.pkl")
joblib.dump(scaler_truth,  "scaler_truth.pkl")

['scaler_truth.pkl']

Definiamo adesso la cartella in cui salvare i file:

In [12]:
output_dir = "/home/christian/Scrivania/notebook/VBS analysis/risultati_predizioni/grid_search"
comparison_dir = "/home/christian/Scrivania/notebook/VBS analysis/risultati_predizioni"

Dobbiamo creare ora una funzione che contenga il nostro modello di rete neurale cosi da poterlo richiamare nel metodo Keras.Regressor e poter provare i diversi parametri di fit.